## Numpy Array Review

## Exercise:
Create the following array

```
[[1, 1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 2],
 [1, 6, 1, 1]]
```

## Exercise:

Given the array below. Attempt to slice the different grouped colors. 

<div align="center"><img src="https://raw.githubusercontent.com/eitanlees/ISC-3313/master/images/numpy_indexing.png" width="%50" /></div>

# Numpy Operations

<div align="center"><img src="https://raw.githubusercontent.com/eitanlees/ISC-3313/master/images/cotton_mill.gif" width="%40"/></div>

## The Slowness of Loops

Python's default implementation (known as CPython) does some operations very slowly. 

This is in part due to the dynamic, interpreted nature of the language: the fact that types are flexible, so that sequences of operations cannot be compiled down to efficient machine code as in languages like C and Fortran. 

The relative sluggishness of Python generally manifests itself in situations where many small operations are being repeated - for instance looping over arrays to operate on each element. 

For example, imagine we have an array of values and we'd like to compute the reciprocal of each. 

A straightforward approach might look like this:

In [2]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

If we measure the execution time of this code for a large input, we see that this operation is very slow, perhaps surprisingly so! 

We'll benchmark this with IPython's `%timeit` magic

In [7]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

3.7 s ± 52.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


It takes several seconds to compute these million operations and to store the result! 

When even cell phones have processing speeds measured in Giga-FLOPS (i.e., billions of numerical operations per second), this seems almost absurdly slow. 

It turns out that the bottleneck here is not the operations themselves, but the type-checking and function dispatches that CPython must do at each cycle of the loop. 

Each time the reciprocal is computed, Python first examines the object's type and does a dynamic lookup of the correct function to use for that type. 

If we were working in compiled code instead, this type specification would be known before the code executes and the result could be computed much more efficiently.

## Introducing UFuncs

For many types of operations, NumPy provides a convenient interface into just this kind of statically typed, compiled routine. 

This is known as a vectorized operation. 

This can be accomplished by simply performing an operation on the array, which will then be applied to each element. 

This vectorized approach is designed to push the loop into the compiled layer that underlies NumPy, leading to much faster execution.

Compare the results of the following two:

In [8]:
print(compute_reciprocals(values))
print(1.0 / values)

[0.16666667 1.         0.25       0.25       0.125     ]
[0.16666667 1.         0.25       0.25       0.125     ]


Looking at the execution time for our big array, we see that it completes orders of magnitude faster than the Python loop:

In [9]:
%timeit (1.0 / big_array)

2.96 ms ± 33.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Vectorized operations in NumPy are implemented via ufuncs, whose main purpose is to quickly execute repeated operations on values in NumPy arrays. 

Ufuncs are extremely flexible – before we saw an operation between a scalar and an array, but we can also operate between two arrays:

In [10]:
np.arange(5) / np.arange(1, 6)

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

And ufunc operations are not limited to one-dimensional arrays–they can also act on multi-dimensional arrays as well:

In [14]:
x = np.arange(9).reshape((3, 3))
5 * x 

array([[ 0,  5, 10],
       [15, 20, 25],
       [30, 35, 40]])

Computations using vectorization through ufuncs are nearly always more efficient than their counterpart implemented using Python loops, especially as the arrays grow in size.

Any time you see such a loop in a Python script, you should consider whether it can be replaced with a vectorized expression.

## Exercise

Evaluate the expression:
$ 3a + b,$

where $a = [1, 2, 3, 4, 5]$ and $b = [e^{-0.1}, e^{-0.2}, e^{-0.3}, e^{-0.4}, e^{-0.5}].$ 

Use `linspace` and/or `arrange` to create $a$ and $b$.

# Aggregations: Min, Max, and Everything In Between

Often when faced with a large amount of data, a first step is to compute summary statistics for the data in question.


NumPy has fast built-in aggregation functions for working on arrays; we'll discuss and demonstrate some of them here.

## Summing the Values in an Array

As a quick example, consider computing the sum of all values in an array.
Python itself can do this using the built-in ``sum`` function:

In [15]:
import numpy as np
np.random.seed(42)

L = np.random.random(100)
sum(L)

47.01807433782094

The syntax is quite similar to that of NumPy's ``sum`` function, and the result is the same in the simplest case:

In [6]:
np.sum(L)

47.018074337820934

However, because it executes the operation in compiled code, NumPy's version of the operation is computed much more quickly:

In [8]:
big_array = np.random.rand(1000000)
%timeit sum(big_array)
%timeit np.sum(big_array)

109 ms ± 836 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
473 µs ± 5.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Be careful, though: the ``sum`` function and the ``np.sum`` function are not identical, which can sometimes lead to confusion!

In particular, their optional arguments have different meanings, and ``np.sum`` is aware of multiple array dimensions, as we will see in the following section.

## Minimum and Maximum

Similarly, Python has built-in ``min`` and ``max`` functions, used to find the minimum value and maximum value of any given array:

In [9]:
min(big_array), max(big_array)

(5.188445665327279e-07, 0.9999983148609545)

NumPy's corresponding functions have similar syntax, and again operate much more quickly:

In [10]:
np.min(big_array), np.max(big_array)

(5.188445665327279e-07, 0.9999983148609545)

In [11]:
%timeit min(big_array)
%timeit np.min(big_array)

82.1 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
461 µs ± 8.89 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


For ``min``, ``max``, ``sum``, and several other NumPy aggregates, a shorter syntax is to use methods of the array object itself:

In [8]:
print(big_array.min(), big_array.max(), big_array.sum())

1.17171281366e-06 0.999997678497 499911.628197


Whenever possible, make sure that you are using the NumPy version of these aggregates when operating on NumPy arrays!

### Multi dimensional aggregates

One common type of aggregation operation is an aggregate along a row or column.
Say you have some data stored in a two-dimensional array:

In [16]:
M = np.random.random((3, 4))
print(M)

[[0.03142919 0.63641041 0.31435598 0.50857069]
 [0.90756647 0.24929223 0.41038292 0.75555114]
 [0.22879817 0.07697991 0.28975145 0.16122129]]


By default, each NumPy aggregation function will return the aggregate over the entire array:

In [17]:
M.sum()

4.570309849333379

Aggregation functions take an additional argument specifying the *axis* along which the aggregate is computed. 

For example, we can find the minimum value within each column by specifying ``axis=0``:

In [15]:
M.min(axis=0)

array([0.1266411 , 0.08740203, 0.43253288, 0.32867611])

The function returns four values, corresponding to the four columns of numbers.

Similarly, we can find the maximum value within each row:

In [13]:
M.max(axis=1)

array([0.5978419 , 0.78922956, 0.88071207])

<div align="center"><img src="https://raw.githubusercontent.com/eitanlees/ISC-3313/master/images/nd_array.png" width="%80"/></div>

The way the axis is specified here can be confusing to users coming from other languages.

The ``axis`` keyword specifies the *dimension of the array that will be collapsed*, rather than the dimension that will be returned.

So specifying ``axis=0`` means that the first axis will be collapsed: for two-dimensional arrays, this means that values within each column will be aggregated.

## Exercise

Consider the heights of US presidents in centimeters given below. 

Calculate the mean, standard deviation, min and max of the data set. 

In [32]:
heights = np.array([189, 170, 189, 163, 183, 171, 185, 168, 173, 183, 173, 173, 175,
       178, 183, 193, 178, 173, 174, 183, 183, 168, 170, 178, 182, 180,
       183, 178, 182, 188, 175, 179, 183, 193, 182, 183, 177, 185, 188,
       188, 182, 185])

## Exercise

Calculate the total sum, row sum, and column sum of the following matrix

In [36]:
np.random.seed(42)

A = np.random.randint(1, 101, size=(5, 3))

print(A)

[[ 52  93  15]
 [ 72  61  21]
 [ 83  87  75]
 [ 75  88 100]
 [ 24   3  22]]
